In [31]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [32]:
df = pd.read_csv('sol.csv')
df.head(5) 

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2020-04-10 00:00:00+00:00,0.832005,1.313487,0.694187,0.951054,87364276,0.0,0.0
1,2020-04-11 00:00:00+00:00,0.951054,1.049073,0.765020,0.776819,43862444,0.0,0.0
2,2020-04-12 00:00:00+00:00,0.785448,0.956670,0.762426,0.882507,38736897,0.0,0.0
3,2020-04-13 00:00:00+00:00,0.890760,0.891603,0.773976,0.777832,18211285,0.0,0.0
4,2020-04-14 00:00:00+00:00,0.777832,0.796472,0.628169,0.661925,16747614,0.0,0.0


In [33]:
df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock Splits'],
      dtype='object')

In [34]:
df.drop([ 'Open', 'High', 'Low',  'Dividends', 'Stock Splits', 'Volume'], inplace = True, axis = 1)

In [35]:
df['time'] = pd.to_datetime(df['Date'])
df['formatted_time'] = df['time'].dt.strftime('%d-%m-%Y')
df['formatted_time']

0       10-04-2020
1       11-04-2020
2       12-04-2020
3       13-04-2020
4       14-04-2020
           ...    
1511    30-05-2024
1512    31-05-2024
1513    01-06-2024
1514    02-06-2024
1515    03-06-2024
Name: formatted_time, Length: 1516, dtype: object

In [36]:
df.drop([ 'Date', 'time'], inplace = True, axis = 1)

In [37]:
df

,Close,formatted_time
0,0.951054,10-04-2020
1,0.776819,11-04-2020
2,0.882507,12-04-2020
3,0.777832,13-04-2020
4,0.661925,14-04-2020
...,...,...
1511,166.976532,30-05-2024
1512,165.637711,31-05-2024
1513,166.030670,01-06-2024
1514,163.249908,02-06-2024


In [38]:
# df['Symbol'].unique()

In [39]:
# df[df['Symbol'] == 'BTC']['Close'].plot()

In [40]:
df.describe()

,Close
count,1516.000000
mean,54.675174
std,59.301607
min,0.515273
25%,14.102368
50%,30.028152
75%,91.004284
max,258.934326


In [41]:
# for i in df['Symbol'].unique():
#     print(len(df[df['Symbol'] == i]))

In [42]:
# state = pd.get_dummies(df['Symbol'])
# state

In [43]:
# df = df.drop('Symbol', axis = 1) 
# df = pd.concat([df, state], axis = 1)
# df.head()

In [44]:
# df = df.replace({True : 1, False : 0})

In [45]:
import pandas_ta as ta
 
df['SMA'] = ta.sma(df['Close'], length=5) 
df['EMA'] = ta.ema(df['Close'], length=5) 
df['RSI'] = ta.rsi(df['Close'], length=14) 

In [46]:
df

,Close,formatted_time,SMA,EMA,RSI
0,0.951054,10-04-2020,NaN,NaN,NaN
1,0.776819,11-04-2020,NaN,NaN,NaN
2,0.882507,12-04-2020,NaN,NaN,NaN
3,0.777832,13-04-2020,NaN,NaN,NaN
4,0.661925,14-04-2020,0.810027,0.810027,NaN
...,...,...,...,...,...
1511,166.976532,30-05-2024,167.409421,168.110910,53.379252
1512,165.637711,31-05-2024,167.866782,167.286510,52.208216
1513,166.030670,01-06-2024,167.073688,166.867897,52.537340
1514,163.249908,02-06-2024,165.993500,165.661901,49.917586


In [47]:
df.fillna(0, inplace = True)

In [48]:
new_df = df.copy()

In [49]:
X = new_df.drop(['Close', 'formatted_time'], axis = 1).values
Y = new_df['Close'].values

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [51]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense
from keras.optimizers import Adam


In [52]:
rf = RandomForestRegressor()
 
param_grid = {
    'n_estimators': [50, 100, 150, 200, 250, 300]
}
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3)
grid_search_rf.fit(X_train, y_train)


GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'n_estimators': [50, 100, 150, 200, 250, 300]})

In [53]:
best_rf = grid_search_rf.best_estimator_
best_rf

RandomForestRegressor(n_estimators=200)

In [54]:
y_pred_rf = best_rf.predict(X_test)
y_pred_rf

array([  0.57405698,   3.10504171, 138.92568382,  28.95467189,
        40.55469036,  20.97383992, 220.3524617 ,   2.19698073,
       137.13448524, 195.08235985,  21.1200752 ,   0.58214348,
       138.54089409,   2.53488665,  22.38024187,  39.91091562,
        39.1692704 , 202.49852882, 148.70413422,  33.07166001,
       141.73571373,   1.46241669,  23.37167775,   4.30497154,
        37.53110014,  28.23054494,  31.83923242,   2.07084297,
        14.83935794, 102.36986752,   3.03890539,  87.84394375,
       211.50442307,  29.05755261,  33.54557838,  71.37082535,
        32.15194596,   0.729493  ,  17.17314211,  20.24508462,
        40.29679873,  29.67489667,  65.68267378,  55.67831963,
       137.25177559,  39.41590633,  45.58860067,  14.1998261 ,
        23.29614573,  88.86410805,   0.65105332,  23.94572088,
        11.45443222, 159.91366852,  26.40606418, 163.62478981,
        28.58306169,  33.69223026,  31.50003181,  60.1520764 ,
        13.88794511,   3.20666787,  55.98336912,  21.89

In [55]:
rf_metrics = {
    'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_rf)),
    'MSE': mean_squared_error(y_test, y_pred_rf),
    'MAE': mean_absolute_error(y_test, y_pred_rf),
    'R2': r2_score(y_test, y_pred_rf)
}
print(rf_metrics)


{'RMSE': 2.66639157185031, 'MSE': 7.109644014434368, 'MAE': 1.3130625530969542, 'R2': 0.9978371641937297}


In [56]:
len(df)

1516

In [57]:
data = df [len(df) - 15:]

In [58]:
for i in range(2):
    sma = ta.sma(data['Close'], length=5).iloc[-1]
    ema = ta.ema(data['Close'], length=5).iloc[-1]
    rsi = ta.rsi(data['Close'], length=14).iloc[-1]
    # sma_5, ema_5, rsi_14 = scaler.transform(np.array([sma, ema, rsi]).reshape(-1, 1))
    # new_close = best_rf.predict([[ sma_5[0], ema_5[0], rsi_14[0]]])
    new_close = best_rf.predict([[sma, ema, rsi]])
    print(new_close)
    new_row = pd.DataFrame({"Close": new_close, "formatted_time" : "0", "SMA" : sma, "EMA" : ema, "RSI" : rsi})
    data = pd.concat([data, new_row], ignore_index= True)
    

[154.53660736]
[152.67962601]


In [59]:

# Evaluate the model
rf_metrics = {
    'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_rf)),
    'MSE': mean_squared_error(y_test, y_pred_rf),
    'MAE': mean_absolute_error(y_test, y_pred_rf),
    'R2': r2_score(y_test, y_pred_rf)
}
print(rf_metrics)


{'RMSE': 2.66639157185031, 'MSE': 7.109644014434368, 'MAE': 1.3130625530969542, 'R2': 0.9978371641937297}


In [60]:
import pickle
with open('sol.pkl', 'wb') as f:
    pickle.dump(best_rf, f)